# Simple Pendulum Simulation

Interactive simulation of a simple pendulum using the myASC-ODE framework.

This example demonstrates the basic setup and simulation of a pendulum system with:
- Mass-spring constraints
- Gravitational forces  
- Real-time 3D visualization using pythreejs
- Physics-based motion analysis

In [35]:
import sys
import os


try:

    current_dir = os.path.dirname(os.path.realpath(__file__))
except NameError:

    current_dir = os.getcwd()


project_root = os.path.join(current_dir, '..', '..')
mechsystem_path = os.path.join(project_root, 'mechsystem')
build_path = os.path.join(project_root, 'build', 'mechsystem')

sys.path.insert(0, mechsystem_path)
sys.path.insert(0, build_path)


sys.path.append('/Users/constantinpierer/Documents/TU_Wien/Technische_Mathematik/7.Semester/Repositorys/myASC-ODE/mechsystem')
sys.path.append('/Users/constantinpierer/Documents/TU_Wien/Technische_Mathematik/7.Semester/Repositorys/myASC-ODE/build/mechsystem')



from mass_spring import *
from pythreejs import *

import numpy as np
import matplotlib.pyplot as plt
from time import sleep

In [36]:
mss = MassSpringSystem3d()
mss.gravity = (0,0,-9.81)

fU = mss.add (Fix((0,0,0)))

m1 = mss.add (Mass(100, (0,1,0),(0,0,0)))
m2 = mss.add (Mass(100,(0,2,0),(0,0,0)))

mss.add (Joint(sqrt(1),(m1, fU)))
mss.add (Joint(sqrt(1),(m1, m2)))

1

In [37]:
masses = []
for m in mss.masses:
    masses.append(
        Mesh(SphereBufferGeometry(0.2, 16, 16),
             MeshStandardMaterial(color='red'),
             position=(m.pos[0], m.pos[1], m.pos[2])))

fixes = []
for f in mss.fixes:
    fixes.append(
        Mesh(SphereBufferGeometry(0.2, 32, 16),
             MeshStandardMaterial(color='blue'),
             position=(f.pos[0], f.pos[1], f.pos[2])))

springpos = []
for s in mss.springs:
    pA = mss[s.connectors[0]].pos
    pB = mss[s.connectors[1]].pos
    springpos.append ([ [pA[0], pA[1], pA[2]], [pB[0], pB[1], pB[2]] ])

if springpos:
    springgeo = LineSegmentsGeometry(positions=springpos)
    m2 = LineMaterial(linewidth=3, color='cyan')
    springs = LineSegments2(springgeo, m2)    

jointpos = []
for b in mss.joints:
    pA = mss[b.connectors[0]].pos
    pB = mss[b.connectors[1]].pos
    jointpos.append ([ [pA[0], pA[1], pA[2]], [pB[0], pB[1], pB[2]] ])

if jointpos:
    jointgeo = LineSegmentsGeometry(positions=jointpos)
    m2 = LineMaterial(linewidth=4, color='green')
    joints = LineSegments2(jointgeo, m2)    


axes = AxesHelper(1)

In [38]:
view_width = 1000
view_height = 800

camera = PerspectiveCamera( position=[10, 6, 10], aspect=view_width/view_height)
key_light = DirectionalLight(position=[0, 10, 10])
ambient_light = AmbientLight()

scene = Scene(children=[*masses, *fixes, axes, camera, key_light, ambient_light] + ([] if not jointgeo else [joints]) + ([] if not springpos else [springs]) )
controller = OrbitControls(controlling=camera)
renderer = Renderer(camera=camera, scene=scene, controls=[controller],
                    width=view_width, height=view_height)

renderer

Renderer(camera=PerspectiveCamera(aspect=1.25, position=(10.0, 6.0, 10.0), projectionMatrix=(1.0, 0.0, 0.0, 0.…

In [ ]:
from time import sleep
for i in range(100):
    mss.simulate ( 0.02, 20)
    for m,mvis in zip(mss.masses, masses):
        mvis.position = (m.pos[0], m.pos[1], m.pos[2])

    springpos = []
    for s in mss.springs:
        pA = mss[s.connectors[0]].pos
        pB = mss[s.connectors[1]].pos
        springpos.append ([ [pA[0], pA[1], pA[2]], [pB[0], pB[1], pB[2]] ])
    if springpos:
        springs.geometry = LineSegmentsGeometry(positions=springpos)

    jointpos = []
    for b in mss.joints:
        pA = mss[b.connectors[0]].pos
        pB = mss[b.connectors[1]].pos
        jointpos.append ([ [pA[0], pA[1], pA[2]], [pB[0], pB[1], pB[2]] ])
    if jointpos:
        joints.geometry = LineSegmentsGeometry(positions=jointpos)

    sleep(0.01)